In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties
sns.set(font=['sans-serif'])
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})
# scikit-learn
from sklearn import datasets, linear_model, preprocessing, neighbors
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import PowerTransformer

In [2]:
# load data
df= pd.read_csv('dataset_all_original.csv', engine='python')

In [3]:
df.head(5)

,Unnamed: 0.1,Unnamed: 0,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,...,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,城市代碼,車位移轉總面積平方公尺,建案名稱,棟及號
0,0,1,文山區,房地(土地+建物),臺北市文山區景後街９５號五樓之１,4.07,商,NaN,NaN,1050418,...,RPTNMLNJNHLFFAA47CA,21.97,0.00,2.57,NaN,NaN,A,NaN,NaN,NaN
1,1,2,文山區,房地(土地+建物),臺北市文山區景文街76號7樓,9.54,商,NaN,NaN,1050327,...,RPUOMLPJNHLFFAA87CA,62.32,8.05,6.22,NaN,NaN,A,NaN,NaN,NaN
2,2,3,文山區,房地(土地+建物),臺北市文山區木柵路四段33巷9號2樓之3,11.53,住,NaN,NaN,1050331,...,RPVOMLPJNHLFFAA97CA,26.01,2.13,0.00,NaN,NaN,A,NaN,NaN,NaN
3,3,4,文山區,房地(土地+建物)+車位,臺北市文山區興隆路三段181巷72號9樓,52.84,住,NaN,NaN,1050327,...,RPVNMLNKNHLFFAA77CA,102.64,4.67,10.95,NaN,NaN,A,NaN,NaN,NaN
4,4,5,文山區,房地(土地+建物),臺北市文山區忠順街二段５８號三樓,15.69,住,NaN,NaN,1050502,...,RPPNMLOKNHLFFAA27CA,57.07,0.00,17.15,NaN,NaN,A,NaN,NaN,NaN


In [4]:
# slice datetime
df['交易年月'] = (df['交易年月日']/100).astype('int64')

In [5]:
# select specific columns with fancy index
df_fi = df[["城市代碼",
            "交易標的",
            "建物移轉總面積平方公尺",
            "都市土地使用分區",
            "交易年月",
            "交易筆棟數",
            "移轉層次",
            "總樓層數",
            "建物型態",
            "主要用途",
            "主要建材",
            "建築完成年月",
            "建物現況格局-房",
            "建物現況格局-廳",
            "建物現況格局-衛",
            "有無管理組織",
            "總價元",
            "單價元平方公尺" ,
            "車位總價元"]]

In [6]:
df_fi.head(5)

,城市代碼,交易標的,建物移轉總面積平方公尺,都市土地使用分區,交易年月,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,有無管理組織,總價元,單價元平方公尺,車位總價元
0,A,房地(土地+建物),35.56,商,10504,土地1建物1車位0,五層,十三層,套房(1房1廳1衛),見使用執照,鋼筋混凝土造,831108,1,1,1,有,5750000,161699.0,0
1,A,房地(土地+建物),115.48,商,10503,土地1建物1車位0,七層,三十三層,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,1040427,2,2,1,有,22600000,195705.0,0
2,A,房地(土地+建物),41.01,住,10503,土地1建物1車位0,二層,十層,套房(1房1廳1衛),住家用,鋼筋混凝土造,971208,1,0,1,有,6000000,146306.0,0
3,A,房地(土地+建物)+車位,219.08,住,10503,土地3建物1車位1,九層,十二層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,1001024,3,2,2,有,29200000,133285.0,0
4,A,房地(土地+建物),83.37,住,10505,土地1建物1車位0,三層,七層,華廈(10層含以下有電梯),見使用執照,鋼筋混凝土造,730712,3,2,2,有,8000000,95958.0,0


In [7]:
df_fi.isna().sum()

城市代碼                 0
交易標的                 2
建物移轉總面積平方公尺          0
都市土地使用分區        698396
交易年月                 0
交易筆棟數                0
移轉層次            941335
總樓層數            945158
建物型態                 0
主要用途            996352
主要建材            942603
建築完成年月         1286350
建物現況格局-房             0
建物現況格局-廳             0
建物現況格局-衛             0
有無管理組織               0
總價元                  0
單價元平方公尺          68149
車位總價元                0
dtype: int64

In [8]:
# remove garage & land & NaN columns
con1 = ~(df_fi["交易標的"] == "車位")
con2 = ~(df_fi["交易標的"] == "土地")
df_no_gar_land = df_fi[con1 & con2]
df_dropna = df_no_gar_land.dropna()
df_dropna=df_dropna.reset_index(drop=True)
# df_dropna

In [9]:
# 篩選"建物型態"
df_type=df_dropna.copy()
df_type['建物型態']=df_type[~df_type['建物型態'].isin(['店面(店鋪)','辦公商業大樓','其他','廠辦'])]['建物型態']
df_type=df_type.reset_index(drop=True)
# df_type

In [10]:
# 篩選"土地使用分區"
df_type.loc[df_type['都市土地使用分區'].str.contains('住'),'都市土地使用分區']='住'
df_type.loc[df_type['都市土地使用分區'].str.contains('商'),'都市土地使用分區']='商'
df_landuse=df_type[~df_type['都市土地使用分區'].str.contains('都市')]
df_landuse=df_landuse.reset_index(drop=True)
df_landuse=df_landuse.drop(df_landuse[(df_landuse['都市土地使用分區']=='農') | (df_landuse['都市土地使用分區']=='工')].index)
df_landuse=df_landuse.reset_index(drop=True)
df_landuse['都市土地使用分區'].value_counts()

住     1603055
商      354506
其他      58890
Name: 都市土地使用分區, dtype: int64

In [11]:
# 篩選"主要建材"
# df_landuse['主要建材'].value_counts()>1000
df_ing=df_landuse.groupby("主要建材").filter(lambda ing: len(ing) > 1000)
df_ing["主要建材"].value_counts()

鋼筋混凝土造       1655501
見其他登記事項       109933
加強磚造           87665
鋼筋混凝土構造        50131
鋼骨鋼筋混凝土造       29018
鋼筋混凝土加強磚造      24700
見其它登記事項        23523
鋼筋混凝土          16139
磚造              4137
見使用執照           2820
鋼骨混凝土造          1954
ＲＣ造             1530
鋼骨造             1505
木造              1307
鋼骨構造            1142
Name: 主要建材, dtype: int64

In [12]:
# delete useless floor
useless_floor = df_ing["移轉層次"].str.len() > 6
df_floorclr = df_ing[~useless_floor]

# modify floor value
df_floormod=df_floorclr.copy()
df_floormod["移轉層次"] = df_floormod["移轉層次"].str[:2]

# select count value > 500
df_floormod = df_floormod.groupby("移轉層次").filter(lambda grp: len(grp) > 500)

# 刪除"移轉層次"與"總樓層數"不合理欄位
mask_balcony=df_floormod['移轉層次']=='陽台'
mask_under=df_floormod['移轉層次']=='地下'
mask_space=df_floormod['總樓層數']=='(空白)'
mask_other=df_floormod['總樓層數']=='見其他登記事項'
mask_licence=df_floormod['總樓層數']=='見使用執照'
mask_regist=df_floormod['總樓層數']=='社登'
df_floor_all=df_floormod.drop(df_floormod[mask_balcony | mask_under | mask_space | mask_other | mask_licence | mask_regist].index)
df_floor_all=df_floor_all.reset_index(drop=True)
# df_floor_all

In [13]:
# 刪除只有"衛"的欄位
df_mask1=df_floor_all["建物現況格局-房"]==0
df_mask2=df_floor_all["建物現況格局-廳"]==0
df_mask3=df_floor_all["建物現況格局-衛"]==1
df_pattern_del_001=df_floor_all.drop(df_floor_all[df_mask1 & df_mask2 & df_mask3].index)
df_pattern_del_001=df_pattern_del_001.reset_index(drop=True)
# df_pattern_del_001

In [14]:
# 將房廳衛000設為111
df_mask4=df_pattern_del_001["建物現況格局-房"]==0
df_mask5=df_pattern_del_001["建物現況格局-廳"]==0
df_mask6=df_pattern_del_001["建物現況格局-衛"]==0
df_pattern_del_001.loc[df_mask4 & df_mask5 & df_mask6, ["建物現況格局-房","建物現況格局-廳","建物現況格局-衛"]]=1
df_pattern_del_111=df_pattern_del_001
# df_pattern_del_111

In [15]:
# 計算每坪價格
df_cal=df_pattern_del_111.copy()
df_cal['總價元']=df_cal['總價元']/10000
df_cal['建物移轉總面積坪']=df_cal['建物移轉總面積平方公尺']*0.3025
df_cal['每坪價格']=(df_cal['總價元'])/df_cal['建物移轉總面積坪']

In [16]:
# separate 交易筆棟數
df_sep=df_cal.copy()
a=df_cal["交易筆棟數"].str.split("土地|建物|車位", expand=True)
df_sep["土地"]=a[1].astype('int64')
df_sep["建物"]=a[2].astype('int64')
df_sep["車位"]=a[3].astype('int64')
# df_sep

In [17]:
# 將土地建物車位轉為數值型態
df_sep_num=df_sep.copy()
df_sep_num[["土地","建物","車位"]]=df_sep_num[["土地","建物","車位"]].astype('int64')

In [18]:
# 移除符號
column=df_sep_num['建築完成年月']
mask_dash=column.str.contains(' ') | column.str.contains('-') | column.str.contains('/')
df_del_dash=df_sep_num.drop(df_sep_num[mask_dash].index)
df_del_dash=df_del_dash.reset_index(drop=True)
# df_del_dash

In [19]:
# 計算屋齡
df_del_dash['交易年'] = (df_del_dash['交易年月']/100).astype('int64')
df_del_dash['建築完成年'] = df_del_dash['建築完成年月'].astype('float64')//10000
df_del_dash['屋齡']=(df_del_dash['交易年']-df_del_dash['建築完成年']).astype('int64')
mask_year=df_del_dash['屋齡']<-10
df_del_year=df_del_dash.drop(df_del_dash[mask_year].index)
df_del_year=df_del_year.reset_index(drop=True)
# df_del_year

In [20]:
# 篩選每坪價格<10萬 & >1000萬
# mask_less10=df_del_year["每坪價格"]<10
mask_more1000=df_del_year["每坪價格"]>1000
df_main=df_del_year.drop(df_del_year[mask_more1000].index)
df_main=df_main.reset_index(drop=True)
# df_main

In [21]:
# 新增房間數 (因房廳衛相關性高故保留房=房間數)
df_main_room=df_main.copy()
df_main_room["房間數"]=df_main_room["建物現況格局-房"]
df_main_room=df_main_room.drop(["建物現況格局-房","建物現況格局-廳","建物現況格局-衛"],axis=1)
df_main_room=df_main_room.reset_index(drop=True)

In [22]:
# 篩選"主要用途"含"住"
df_main_living=df_main_room.loc[df_main_room["主要用途"].str.contains('住')]
df_main_living=df_main_living.reset_index(drop=True)
# df_main_living["主要用途"].value_counts()

In [23]:
# 篩選"主要用途"移除有複雜說明欄位
df_main_usage=df_main_living[df_main_living['主要用途'].isin(['住家用','國民住宅','集合住宅','住商用'])]
df_main_usage=df_main_usage.reset_index(drop=True)
# df_main_usage

In [24]:
# 移除沒用到的column
df_main_no_encoding = df_main_usage.drop(['交易筆棟數',
                                    '建物移轉總面積平方公尺',
                                    '單價元平方公尺',
                                    '車位總價元',
                                    '建築完成年月',
                                    '建築完成年',
                                    '總價元',
                                    '交易年'], axis=1)
df_main_no_encoding

,城市代碼,交易標的,都市土地使用分區,交易年月,移轉層次,總樓層數,建物型態,主要用途,主要建材,有無管理組織,建物移轉總面積坪,每坪價格,土地,建物,車位,屋齡,房間數
0,A,房地(土地+建物),商,10503,七層,三十三層,住宅大樓(11層含以上有電梯),住家用,鋼骨鋼筋混凝土造,有,34.932700,64.695829,1,1,0,1,2
1,A,房地(土地+建物),住,10503,二層,十層,套房(1房1廳1衛),住家用,鋼筋混凝土造,有,12.405525,48.365547,1,1,0,8,1
2,A,房地(土地+建物)+車位,住,10503,九層,十二層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,66.271700,44.061040,3,1,1,5,3
3,A,房地(土地+建物)+車位,住,10504,四層,五層,華廈(10層含以下有電梯),住家用,鋼筋混凝土造,有,69.039575,50.695561,1,1,1,7,4
4,A,房地(土地+建物),住,10503,九層,十二層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,24.293775,50.218626,1,1,0,34,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362184,X,房地(土地+建物)+車位,住,11010,十二,十四層,住宅大樓(11層含以上有電梯),住家用,鋼筋混凝土造,有,56.494900,10.974442,2,1,1,15,3
1362185,Z,房地(土地+建物),住,10605,全,二層,透天厝,住家用,鋼筋混凝土造,無,38.115000,0.913748,1,1,0,5,4
1362186,Z,房地(土地+建物),住,10601,全,二層,透天厝,住家用,見其他登記事項,無,28.035700,1.212704,1,1,0,106,3
1362187,Z,房地(土地+建物),住,10402,全,三層,透天厝,住家用,鋼筋混凝土造,無,42.767450,11.223489,1,1,0,16,3


In [25]:
df_main_no_encoding.to_csv('all_dataset_no_encoding.csv')